In [ ]:
%cd ..

In [ ]:
!pip install uv

In [ ]:
!uv pip install -r requirements.txt

In [ ]:
from download_models import download_qwen_models

qwen_image_edit_path, qwen_image_path = download_qwen_models()

In [ ]:
import ctypes

try:
    ctypes.CDLL("libcuda.so.1", mode=ctypes.RTLD_GLOBAL)
    print("✅ Successfully loaded real CUDA driver (libcuda.so.1)")
except OSError:
    print("⚠️ WARNING: Could not load libcuda.so.1 manually.")

In [ ]:
from models.lightx2v_models import QwenImage

model = QwenImage(
  model_path=str(qwen_image_path), 
  quantized_model_path= f"{qwen_image_path}/fp8/Qwen-Image-2512-Lightning-4steps-FP8.safetensors", 
  compile=False,
)

In [ ]:
from pathlib import Path
import requests

def download_image(image_url: str, output_path: str, override=False):
  if Path(output_path).exists() and not override:
    return

  response = requests.get(image_url)
  response.raise_for_status()
  with open(output_path, "wb") as f:
    f.write(response.content)


In [ ]:
import io
import time

from PIL import Image

input_image_path_1 = "img_in_1"
output_image ="img_out.png"

download_image(
  "https://plus.unsplash.com/premium_photo-1690407617542-2f210cf20d7e?fm=jpg&q=60&w=3000&ixlib=rb-4.1.0&ixid=M3wxMjA3fDB8MHxzZWFyY2h8MXx8cGVyc29ufGVufDB8fDB8fHww",
   input_image_path_1,
)

t = time.perf_counter()
model.generate(
  prompt="Make this person look like a robot",
  image_paths=[input_image_path_1],
  aspect_ratio="1:1",
  resolution="1K",
  output_path=output_image,
)
print(f"Generated in {time.perf_counter() - t:.2f}s")

webp_img = io.BytesIO()
Image.open(output_image).save(webp_img,format="WEBP")
webp_img.seek(0)
Image.open(webp_img)